# Data Sourcing & Preprocessing

This notebook involves:
1.  **Sourcing data** from `yfinance` and `FRED`.
2.  **Consolidating** the data into a single pandas DataFrame.
3.  **Cleaning** the data by resampling to a weekly frequency and handling missing values.
4.  **Saving** the final, cleaned DataFrame to the `/data` directory.

In [1]:
import pandas as pd
import yfinance as yf
from fredapi import Fred
import os
import sys
from dotenv import load_dotenv

load_dotenv()

True

## Setup: FRED API Key

To download data from the Federal Reserve Economic Data (FRED) database, you need a free API key.

1.  **Obtain a Key**: Visit the [FRED API Key site](https://fred.stlouisfed.org/docs/api/api_key.html) to request one.
2.  **Set Environment Variable**: For security and best practices, save the key as an environment variable named `FRED_API_KEY`. The notebook uses the `dotenv` library to load this variable from a `.env` file in your project's root directory.

In [3]:
api_key = os.environ.get("FRED_API_KEY")
fred = Fred(api_key=api_key)

## 1. Data Sourcing

We will source data from two primary locations:
*   **Yahoo Finance (`yfinance`)**: For market-based Exchange Traded Funds (ETFs) and volatility indices.
*   **Federal Reserve Economic Data (`FRED`)**: For key economic indicators.

### Ticker Definitions

The following tickers have been selected to provide a comprehensive view of the market environment:

*   **`SPY` (S&P 500 ETF)**: Represents the broader US stock market performance.
*   **`GLD` (Gold ETF)**: A common safe-haven asset and inflation hedge.
*   **`TLT` (20+ Year Treasury Bond ETF)**: Represents long-term US interest rate movements.
*   **`UUP` (US Dollar Index Bullish Fund)**: Tracks the value of the US dollar against a basket of foreign currencies.
*   **`^VIX` (CBOE Volatility Index)**: The "fear index," measuring expected 30-day volatility of the US stock market.
*   **`^MOVE` (ICE BofA MOVE Index)**: Measures volatility in the US Treasury bond market.
*   **`T10Y2Y` (10-Year vs. 2-Year Treasury Yield Spread)**: A key indicator of the yield curve, often used to predict economic recessions.
*   **`BAMLC0A0CMEY` (ICE BofA US Corporate Index Yield)**: Tracks the yield on US investment-grade corporate bonds.
*   **`USRECP` (NBER Recession Indicator)**: A binary indicator (1 for recession, 0 for expansion) from the National Bureau of Economic Research.

In [4]:
# Define tickers for yfinance
yf_tickers = [
    "SPY",  # S&P 500 ETF
    "GLD",  # Gold ETF
    "TLT",  # 20+ Year Treasury Bond ETF
    "UUP",  # US Dollar Index Bullish Fund
    "^VIX",  # Stock Market Volatility Index
    "^MOVE",  # Bond Market Volatility Index
]

# Define series IDs for FRED
fred_tickers = {
    "T10Y2Y": "T10Y2Y",  # 10Y-2Y Yield Curve Spread
    "BAMLC0A0CMEY": "BAMLC0A0CMEY",  # US Corporate Bond Yield
    "USRECP": "USRECP",  # NBER Recession Indicator
}

# Download data using yfinance
print(f"Downloading data for: {', '.join(yf_tickers)}...")
try:
    yf_data = yf.download(yf_tickers, start="2000-01-01", auto_adjust=False)["Close"]  # type: ignore
    # Check if the download returned an empty dataframe, which can happen with errors
    if yf_data.empty:
        raise ValueError(
            "yfinance returned an empty DataFrame. Check tickers or network connection."
        )
except Exception as e:
    print(f"An error occurred during yfinance download: {e}")
    # Exit or handle the error appropriately. Exiting is often safest in a data pipeline.
    sys.exit("Stopping execution due to data download failure.")

print("yfinance data downloaded successfully.")


# Download data using fredapi
print(f"Downloading data for: {', '.join(fred_tickers.keys())}...")
fred_data = pd.DataFrame()
for key, series_id in fred_tickers.items():
    fred_data[key] = fred.get_series(series_id, start="2000-01-01")
print("FRED data downloaded successfully.")

[*********************100%***********************]  6 of 6 completed


yfinance data downloaded successfully.
FRED data downloaded successfully.


## 2. Data Consolidation and Cleaning

Once sourced, the data from `yfinance` and `FRED` is combined into a single time-series DataFrame. The following preprocessing steps are then applied:

1.  **Consolidation**: The two datasets are merged based on their date index. An `outer` join is used to ensure no data is lost if dates don't perfectly align.
2.  **Resampling to Weekly Frequency**: Financial data is noisy. To focus on meaningful trends, we resample the daily data to a weekly frequency (`'W-FRI'`), taking the last recorded value of each week (Friday).
3.  **Forward Fill Missing Values**: Not all markets are open on the same days, and some data is released less frequently. We use forward-fill (`ffill`) to propagate the last known value forward, ensuring a complete time series.
4.  **Drop Initial NaNs**: Any remaining `NaN` values, which typically occur at the beginning of the dataset before all series have started, are dropped.

In [5]:
# Combine the two dataframes
all_data = pd.concat([yf_data, fred_data], axis=1, join="outer")

# Normalize all data to a weekly frequency, taking the last value of the week (Friday)
weekly_data = all_data.resample("W-FRI").last()

# Handle missing values - a common issue with financial data
weekly_data.ffill(inplace=True)

# It's good practice to drop any remaining NaNs, which might be at the very beginning
weekly_data.dropna(inplace=True)

print("Data consolidation and cleaning complete.")

Data consolidation and cleaning complete.


## 3. Save the Final DataFrame

In [6]:
# Create the data directory if it doesn't exist
data_dir = "../data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Save the cleaned data to a CSV file for easy access in the next phase
output_path = os.path.join(data_dir, "cleaned_market_data.csv")
weekly_data.to_csv(output_path)

print(f"Final, cleaned data saved to: {output_path}")


# --- 4. Inspect the Results ---

print("\n--- Final DataFrame ---")
print(
    f"Date Range: {weekly_data.index.min().date()} to {weekly_data.index.max().date()}"
)
print(f"Shape: {weekly_data.shape}")
print("\nHead:")
print(weekly_data.head())
print("\nTail:")
print(weekly_data.tail())
print("\nData Types and Non-Null Values:")
weekly_data.info()

Final, cleaned data saved to: ../data\cleaned_market_data.csv

--- Final DataFrame ---
Date Range: 2007-03-02 to 2025-07-18
Shape: (960, 9)

Head:
                  GLD         SPY        TLT        UUP      ^MOVE       ^VIX  \
2007-03-02  63.709999  138.669998  90.199997  24.959999  76.699997  18.610001   
2007-03-09  64.250000  140.779999  89.400002  25.160000  63.599998  14.090000   
2007-03-16  64.620003  138.529999  89.849998  24.870001  67.000000  16.790001   
2007-03-23  65.150002  143.389999  88.790001  24.930000  68.699997  12.950000   
2007-03-30  65.739998  142.000000  88.279999  24.790001  67.900002  14.640000   

            T10Y2Y  BAMLC0A0CMEY  USRECP  
2007-03-02   -0.04          5.46     0.0  
2007-03-09   -0.07          5.55     0.0  
2007-03-16   -0.03          5.52     0.0  
2007-03-23    0.02          5.58     0.0  
2007-03-30    0.07          5.60     0.0  

Tail:
                   GLD         SPY        TLT        UUP      ^MOVE  \
2025-06-20  310.130005  594.28